In [70]:
import os
import pandas as pd
import matplotlib.pyplot as plt

country_list = {
    '미국': 'USD',
    '유럽연합': 'EUR',
    '일본': 'JPY',
    '중국': 'CNY',
    '홍콩': 'HKD',
    '대만': 'TWD',
    '영국': 'GBP',
    '오만': 'OMR',
    '캐나다': 'CAD',
    '스위스': 'CHF',
    '스웨덴': 'SEK',
    '호주': 'AUD',
    '뉴질랜드':'NZD',
    '체코': 'CZK',
    '칠레': 'CLP',
    '터키': 'TRY',
    '몽골': 'MNT',
    '이스라엘': 'ILS',
    '덴마크': 'DKK',
    '노르웨이': 'NOK',
    '사우디아라비아': 'SAR',
    '쿠웨이트': 'KWD',
    '바레인': 'BHD',
    '아랍에미리트': 'AED',
    '요르단': 'JOD',
    '이집트': 'EGP',
    '태국': 'THB',
    '싱가포르': 'SGD',
    '말레이시아': 'MYR',
    '인도네시아': 'IDR',
    '카타르': 'QAR',
    '카자흐스탄': 'KZT',
    '브루나이': 'BND',
    '인도': 'INR',
    '파키스탄': 'PKR',
    '방글라데시': 'BDT',
    '필리핀': 'PHP',
    '멕시코': 'MXN',
    '브라질': 'BRL',
    '베트남': 'VND',
    '남아프리카공화국':'ZAR',
    '러시아': 'RUB',
    '헝가리': 'HUF',
    '폴란드': 'PLN'
}
# result 폴더 생성
if not any(['result' == s for s in os.listdir('.')]):
    os.mkdir('./result')

country_list = list(country_list.keys())[:4]
result = list()
print('----- 수집 시작 -----')
for country in country_list:
    index_cd = index_cd_list[country]
    page = 1
    while(True):
        url = 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_{}KRW&page={}'.format(index_cd, page)
        data = pd.read_html(url)[0]
        result.append(data)
        data.insert(0, '화폐구분', index_cd)
        data.insert(0, '국가구분', country)
        if (data.shape[0] != 10):
            break
        page += 1
    print('환율정보 수집 완료(국가구분 : {}, 화폐구분: {}, 페이지 수: {})'.format(country, index_cd, page))
df = pd.concat(result)
df.columns = ['국가구분', '화폐구분', '날짜', '매매기준율', '전일대비', '현찰_사실때', '현찰_파실때', '송금_보내실때', '송금_받으실때', 'T/C_사실때', '외화수표_파실때']
df = df.drop('전일대비', axis=1)
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y.%m.%d')
df = df.reset_index(drop=True)
print('----- 수집 완료 -----')

----- 수집 시작 -----
환율정보 수집 완료(국가구분 : 미국, 화폐구분: USD, 페이지 수: 383)
환율정보 수집 완료(국가구분 : 유럽연합, 화폐구분: EUR, 페이지 수: 383)
환율정보 수집 완료(국가구분 : 일본, 화폐구분: JPY, 페이지 수: 383)
환율정보 수집 완료(국가구분 : 중국, 화폐구분: CNY, 페이지 수: 383)
----- 수집 완료 -----


In [75]:
writer = pd.ExcelWriter('./result/exchange.xlsx', 'xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
writer.close()

In [ ]:
plt.figure(figsize=(12,8))
for _, grp in df.groupby('화폐구분'):
    plt.plot(grp['날짜'], grp['매매기준율'], label=grp['화폐구분'].iloc[0])
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig('./result/exchange_example.png')
plt.show()